In [8]:
import pandas as pd # We use pandas to manipulate the data
import json  #Load the the data in json
from pprint import pprint #Print things pretty
results = json.load(open('es1_annotations.json')) # Our downloaded results
results.keys()

dict_keys(['id', 'examples', 'schema', 'dataset', 'relations', 'name', 'annotators_per_example'])

In [9]:
examples = results['examples']
pd.DataFrame(examples[:2])

,content,seen_by,comments,metadata,example_id,annotations,classifications
0,"Those established for each zone (main, compati...","[{'annotator': 'amna.dridi@bcu.ac.uk', 'annota...",[],{},fe212807-e101-48c0-ae9c-0ea4e62b956c,"[{'end': 31, 'tag': 'Object', 'start': 27, 'va...",[]
1,The unit of parcel resulting from the present ...,"[{'annotator': 'amna.dridi@bcu.ac.uk', 'annota...",[],{},389010ac-8ef9-41a1-a6f2-63c9473c3c6a,"[{'end': 8, 'tag': 'Object', 'start': 4, 'valu...",[]


In [10]:
all_annotations = sum(map(lambda x:x['annotations'],examples),[])

In [11]:
Annotations = pd.DataFrame(all_annotations)
Annotations.pivot_table(index='value',columns='tag',values='example_id',aggfunc=len).fillna(0).head()

tag,Object,Property
value,,
Arabic teula,1.0,0.0
Area,0.0,1.0
Avenue,1.0,0.0
Basement floors,1.0,0.0
Block,1.0,0.0


In [16]:
AnnotatedData = pd.io.json.json_normalize(all_annotations,meta=['tagged_token_id','example_id','tag','value', 'start', 'end'],
                                                          record_path='annotated_by')
AnnotatedData.head()

<ipython-input-16-340047eda73e>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  AnnotatedData = pd.io.json.json_normalize(all_annotations,meta=['tagged_token_id','example_id','tag','value', 'start', 'end'],


,annotator,timestamp,annotator_id,tagged_token_id,example_id,tag,value,start,end
0,amna.dridi@bcu.ac.uk,2023-08-24T17:51:24.81+00:00,1.0,6c25e98d-3b01-4da3-b288-4ac760fa4807,fe212807-e101-48c0-ae9c-0ea4e62b956c,Object,zone,27,31
1,amna.dridi@bcu.ac.uk,2023-08-24T18:32:34.239+00:00,1.0,4f5cb456-22ee-4e18-aff7-47727fda1305,389010ac-8ef9-41a1-a6f2-63c9473c3c6a,Object,unit,4,8
2,amna.dridi@bcu.ac.uk,2023-08-24T18:32:52.953+00:00,1.0,c91494bb-af92-41a5-b6c6-4e03847f790b,389010ac-8ef9-41a1-a6f2-63c9473c3c6a,Object,property,361,369
3,amna.dridi@bcu.ac.uk,2023-08-24T18:32:45.792+00:00,1.0,d9bc5f24-6f2e-4ed0-9f5c-000f2f9caf75,389010ac-8ef9-41a1-a6f2-63c9473c3c6a,Object,urban planning,164,178
4,amna.dridi@bcu.ac.uk,2023-08-24T18:32:39.134+00:00,1.0,a2baf0e5-8f58-4b12-b5f3-cf8922043025,389010ac-8ef9-41a1-a6f2-63c9473c3c6a,Object,parcel,12,18


In [17]:
AnnotatedData = pd.io.json.json_normalize(all_annotations,meta=['value','tag'],
                                                          record_path='annotated_by')
AnnotatedData.head()

<ipython-input-17-fb64854c631c>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  AnnotatedData = pd.io.json.json_normalize(all_annotations,meta=['value','tag'],


,annotator,timestamp,annotator_id,value,tag
0,amna.dridi@bcu.ac.uk,2023-08-24T17:51:24.81+00:00,1.0,zone,Object
1,amna.dridi@bcu.ac.uk,2023-08-24T18:32:34.239+00:00,1.0,unit,Object
2,amna.dridi@bcu.ac.uk,2023-08-24T18:32:52.953+00:00,1.0,property,Object
3,amna.dridi@bcu.ac.uk,2023-08-24T18:32:45.792+00:00,1.0,urban planning,Object
4,amna.dridi@bcu.ac.uk,2023-08-24T18:32:39.134+00:00,1.0,parcel,Object


In [18]:
spans_output = 'es1_tags.csv'
AnnotatedData.to_csv(spans_output, encoding='utf-8', index=False)